In [ ]:
# Установка необходимых библиотек
!pip install transformers datasets torch scikit-learn


In [ ]:
# Импорт библиотек
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
from sklearn.metrics import classification_report
import pandas as pd


In [ ]:
# Определяем устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Инициализация модели и токенайзера
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Загрузка датасета IMDb
data = load_dataset("imdb")

# Преобразование данных
def preprocess_data(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

encoded_data = data.map(preprocess_data, batched=True)
encoded_data = encoded_data.rename_column("label", "labels")
encoded_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Уменьшение размера данных для быстрого обучения
small_train_data = encoded_data["train"].shuffle(seed=42).select(range(1000))
small_test_data = encoded_data["test"].shuffle(seed=42).select(range(200))

# Определение аргументов тренировки
training_args = TrainingArguments(
    output_dir="./bert_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_data,
    eval_dataset=small_test_data,
)

# Обучение модели
#trainer.train()

# Оценка модели
results = trainer.evaluate()
print("Evaluation results:", results)

# Оценка на тестовых данных
predictions = trainer.predict(small_test_data)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1).tolist()
true_labels = small_test_data["labels"].tolist()
texts = data["test"].shuffle(seed=42).select(range(200, 210))["text"]

# Формирование таблицы с текстами, истинными метками и предсказаниями
labels_map = {0: "negative", 1: "positive"}
comparison_data = {
    "Text": texts,
    "True Label": [labels_map[label] for label in true_labels[:10]],
    "Predicted Label": [labels_map[pred] for pred in preds[:10]],
}
comparison_df = pd.DataFrame(comparison_data)

# Отчет о классификации
report = classification_report(true_labels, preds, target_names=["negative", "positive"])
print("Classification Report on Test Data:\n", report)

# Вывод таблицы
print(comparison_df)